# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
parquet_files

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index('ticker')

dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        AdjClose_lag_1 = x['Adj Close'].shift(1)
    )
).assign(
    returns = lambda x: x['Close'] / x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

dd_feat


C:\Users\bisha\AppData\Local\Temp\ipykernel_2248\3410490902.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,AdjClose_lag_1,returns,hi_lo_range
npartitions=60,,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ALDX,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [7]:
# Write your code below.
df = dd_feat.compute()
df['returns_MA10'] = df['returns'].rolling(3).mean()

In [10]:
df.loc['ACN'].tail()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,AdjClose_lag_1,returns,hi_lo_range,returns_MA10
ticker,,,,,,,,,,,,,,
ACN,2020-03-26,153.399994,171.600006,150.000000,171.339996,171.339996,5194500.0,ACN.csv,2020,151.820007,151.820007,0.128573,21.600006,0.062697
ACN,2020-03-27,165.000000,169.289993,160.610001,162.880005,162.880005,3802500.0,ACN.csv,2020,171.339996,171.339996,-0.049375,8.679993,0.015710
ACN,2020-03-30,164.119995,170.369995,163.779999,169.820007,169.820007,2731300.0,ACN.csv,2020,162.880005,162.880005,0.042608,6.589996,0.040602
ACN,2020-03-31,167.429993,169.880005,162.160004,163.259995,163.259995,2963000.0,ACN.csv,2020,169.820007,169.820007,-0.038629,7.720001,-0.015132
ACN,2020-04-01,158.570007,160.820007,152.580002,154.679993,154.679993,2973100.0,ACN.csv,2020,163.259995,163.259995,-0.052554,8.240005,-0.016192


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**Answers to the above questions:**

> It was not strictly necessary to convert from dask to pandas dataframe. The same could have been done using the dask dataframe. However, if the dataset is small enough to fit in the memory, then it's simpler and faster to use pandas dataframe. 
> 
> For large datasets that don't fit into memory, it would certainly be better to do it in dask (instead of pandas) because dask provides distributed and lazy computation which can easily scale up to process very large datasets. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.